### Set user-defined parameters

In [ ]:
### Set user-defined parameters for analysis ###

# Point the settings file to JSON file with analysis parameters.
# This path (and others in this notebook) can be relative or absolute.
#settings_file = '../data/analysis_settings/2021_12_03_reviewerExpts.json'
settings_file = '../data/analysis_settings/FigXX_HaloTagControls_Diffusion_test.json'

# Set plotting and figure saving params
plot_figs = True
plot_all_tracks = False # warning - takes forever!
save_figs = False # Save output figures
save_data = True # Save filtered tracks to see which ones were identified as correlated.

plot_settings = '../src/plotting_settings.py' # File containing matplotlib settings
save_dir_reports = '../reports/figures' # Base directory for saving figures

### Load modules and data

In [ ]:
### Load modules ###

# Uncomment the following two lines for debugging
%load_ext autoreload
%autoreload 2

# Import general Python modules
import os, sys, inspect
import matplotlib
import matplotlib.pylab as plt
import numpy as np
import math
import pandas as pd
import seaborn as sns

# Add source code directory (src) to path to enable user module import
module_dir = '../src'
os.sys.path.insert(0, module_dir)

# Import user modules from source code directory
import parse_trackmate as pt
import correlation_analysis as corr
import diffusion as dif

In [ ]:
### Load the JSON settings file that specifies analysis details ###
# Note: for diffusion analysis, the JSON settings file must contain
# the following keys: 'diff_dim', 'min_averages_for_msd', 'dc_fit_nframes',
# and "frame_interval_sec" (the latter is in seconds)

conditions, params = corr.read_analysis_params(settings_file, 
                                               save_dir_reports, 
                                               print_summary=True)



In [ ]:
### Set up figure save directories and load plotting style ###

save_dir = params['save_dir']
save_dir_data = params['save_dir_data']

if save_figs: # Save figure files
    %matplotlib
    %run $plot_settings save_large
    
    # Make directories for saving figures
    dir_sum_figs = os.path.join(save_dir, 'summary_figures')
    dir_examples = os.path.join(save_dir, 'examples') 
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    if not os.path.exists(dir_sum_figs):
        os.makedirs(dir_sum_figs)
    if not os.path.exists(dir_examples):
        os.makedirs(dir_examples)    
else: # Plot interactively
    %matplotlib
    %run $plot_settings plot_only
    
if save_data: # Save filtered TrackMate trajectories
    if not os.path.exists(save_dir_data):
        os.makedirs(save_dir_data)

In [ ]:
### Parse track data from TrackMate xml files into Pandas dataframes ###

data_parsed = {} # List of dataframes, one dataframe per condition
for condition in conditions:
    print("Now processing condition: " + condition)
    data_parsed[condition] = pt.read_2color_data(conditions[condition],
                                    do_int_analysis=params['do_int_analysis'],
                                    int_settings=params['int_settings'])
print('Done parsing. Data loading is now complete.')

## Beginning of SASPT analysis

In [ ]:
print(data_parsed['HaloTag-1x'].loc[0,'parsed'])

In [ ]:
from saspt import sample_detections, StateArray, RBME

#test_df = data_parsed['HaloTag-1x'].loc[0,'parsed']
#test_df = data_parsed['HaloTag-GST'].loc[0,'parsed']
#test_df = data_parsed['EndoIRE1-NoStress'].loc[0,'parsed']
test_df = data_parsed['EndoIRE1_4h-Tm'].loc[1,'parsed']
#df_new = test_df.rename(columns={"track_ID": "TRACK", "t": "FRAME", 'x':"PX", "y":"PY"})
df_new = test_df.rename(columns={"track_ID": "trajectory", "t": "frame"})


print(df_new)

detections = df_new
settings = dict(
    likelihood_type = RBME,
    pixel_size_um = 1,
    frame_interval = 0.06,
    focal_depth = 1,
    progress_bar = True,
)


"""
detections = sample_detections()

settings = dict(
    likelihood_type = RBME,
    pixel_size_um = 0.122,
    frame_interval = 0.01,
    focal_depth = 0.7,
    progress_bar = True,
)
"""



SA = StateArray.from_detections(detections, **settings)

print(SA)

#naive_occs = SA.naive_occs
#posterior_occs = SA.posterior_occs

SA.plot_occupations('test.png')

#occupations = SA.occupations_dataframe
#print(occupations)

In [ ]:
# Analysis of trajectories pooled by condition
# Export all trajectories as CSV files that can be read by SASPT

csv_organizer_filename = 'organizer.csv'
save_dir_csv = os.path.join(save_dir_data, 'csv_for_saspt')
if not os.path.exists(save_dir_csv):
        os.makedirs(save_dir_csv)
csv_organizer_path = os.path.join(save_dir_csv, csv_organizer_filename)

csv_paths = []
csv_conditions =[]
for condition in data_parsed:
    data = data_parsed[condition]
    for row in data.itertuples(index=False):
        filename_source = row[0]
        color = row[2]
        data_to_save = row[4][['track_ID','t','x','y']]
        #rename columns as needed
        df = data_to_save.rename(columns={"track_ID": "trajectory", "t": "frame"})
        
        csv_filename = filename_source + '.csv'
        csv_path = os.path.join(save_dir_csv,csv_filename)
        df.to_csv(csv_path, index=False)
        
        csv_condition = condition + '_' + color
        csv_paths.append(csv_path)
        csv_conditions.append(csv_condition)

csv_organizer = pd.DataFrame({"filepath":csv_paths, "condition":csv_conditions})
csv_organizer.to_csv(csv_organizer_path, index=False)


In [ ]:
paths = pd.read_csv(csv_organizer_path)
print(paths)

settings = dict(
    likelihood_type = RBME,
    pixel_size_um = 1,
    frame_interval = 0.06,
    focal_depth = 0.7,
    path_col = 'filepath',
    condition_col = 'condition',
    progress_bar = True,
    num_workers = 4,
)

from saspt import StateArrayDataset
SAD = StateArrayDataset.from_kwargs(paths, **settings)

print(SAD)

SAD.posterior_heat_map('posterior_heat_map.png')

In [ ]:
SAD.posterior_line_plot('posterior_line_plot.png')

In [ ]:
posterior_occs, condition_names = SAD.infer_posterior_by_condition('condition')
print(posterior_occs.shape)
print(condition_names)

In [ ]:
#conditions_to_plot = ['EndoIRE1-NoStress_C1','EndoIRE1_4h-Tm_C1', 'HaloTag-1x_C1', 'HaloTag-GST_C1']
#conditions_to_plot = ['HaloTag-1x_C1', 'HaloTag-GST_C1', 'HaloTag-2x-tandem_C1', 'HaloTag_4x_C1']
conditions_to_plot = ['HaloTag-1x_C1', 'HaloTag-GST_C1','EndoIRE1-NoStress_C1','EndoIRE1_4h-Tm_C1']

from saspt import normalize_2d
posterior_occs = normalize_2d(posterior_occs, axis=1)
diff_coefs = SAD.likelihood.diff_coefs
for c in range(posterior_occs.shape[0]):
    if condition_names[c] in conditions_to_plot:
        plt.plot(diff_coefs, posterior_occs[c,:], label=condition_names[c])
plt.xscale('log')
plt.xlabel('Diff. coef. ($\mu$m$^{2}$ s$^{-1}$)')
plt.ylabel('Mean posterior occupation')
plt.ylim((0, plt.ylim()[1]))
plt.legend()
plt.show()